In [1]:
import pandas as pd
from datetime import datetime

In [52]:
# file = 'F:\\_Bounty\\860800150_mayo-clinic-arizona_standardcharges.csv'
file = '860800150_mayo_head.csv'
url = "https://mcorgstatic.blob.core.windows.net/cms-price/860800150_mayo-clinic-arizona_standardcharges.csv"
ccn = '030103'
output_file = 'az_out.csv'

In [35]:
def payer_category(payer_category):
    cats = {
        'payer_specific_negotiated_rate': 'rate', 
        'standard_gross_charge': 'gross', 
        'discounted_cash_price': 'cash', 
        'minimum_negotiated': 'min',
        'maximum_negotiated': 'max'
    }
    
    return cats[payer_category]

In [42]:
with open(file, 'r') as f:
    for i, line in enumerate(f):
        if i == 1:
            hospital_name = line.split('-')[-1].split(',')[0].strip('"')
        elif i == 2:
            updated_date = line.split(':')[-1].split(',')[0].strip('"').strip()
        if i > 2:
            break

updated_date = datetime.strptime(updated_date, '%Y/%m/%d')
        
print(hospital_name, updated_date)
            

 Arizona 2023-03-19 00:00:00


In [27]:
df = pd.read_csv(file)
# Drop non-csv data
df = df[7:]

In [28]:
df = df.rename(columns={
    'code_desc': 'procedure_desc',
    'hcpcs': 'hcpcs_cpt',
    'setting': 'patient_class',
    'contract_name': 'payer_name',
    # 'payer_specific_negotiated_rate': 'payer_rate'
})

In [29]:
df.drop(columns=[
    'dose_type',
    'applicable_percent',
    'count_of_compared_rates',
    'footnote',
    'product_name',
    'implied_quantity',
    'hcpcs_dose'

], inplace=True)

In [30]:
# df = df.head(2)

In [31]:
# df


,procedure_desc,code,rev_code,rev_desc,hcpcs_cpt,code_type,billing_class,patient_class,billed_quantity,modifier,standard_gross_charge,discounted_cash_price,payer_name,payer_specific_negotiated_rate,minimum_negotiated,maximum_negotiated,rate_method,rate_desc
7,Liver Transplant And/Or Intestinal Transplant,'001-1,NaN,NaN,NaN,APR-DRG,Facility,Inpatient,NaN,NaN,NaN,NaN,1196 AZ MEDICAID PPS,"$35,821.79","$35,821.79","$35,821.79",APR-DRG,NaN
8,LIVER TRANSPLANT AND/OR INTESTINAL TRANSPLANT,'001-1,NaN,NaN,NaN,APR-DRG,Facility,Inpatient,NaN,NaN,NaN,NaN,1215_BLUE CROSS BLUE SHIELD_EMERGENCY SERVICES,"$55,579.42",NaN,NaN,APR-DRG,NaN


In [32]:
cols = df.columns.tolist()
id_vars = cols[:cols.index('modifier')+1]
id_vars.extend(['payer_name', 'rate_method', 'rate_desc'])
print(id_vars)

payer_cat = ['payer_specific_negotiated_rate', 'standard_gross_charge', 'discounted_cash_price', 'minimum_negotiated', 'maximum_negotiated']
df = pd.melt(df, id_vars=id_vars, value_vars=payer_cat, var_name='payer_desc', value_name='rate')

['procedure_desc', 'code', 'rev_code', 'rev_desc', 'hcpcs_cpt', 'code_type', 'billing_class', 'patient_class', 'billed_quantity', 'modifier', 'payer_name', 'rate_method', 'rate_desc']


In [33]:
df_lowers = df[['code_type', 'billing_class', 'patient_class', 'rate_method']]
df[df_lowers.columns] = df_lowers.apply(lambda x: x.str.lower())

In [37]:
df["payer_category"] = df["payer_desc"].apply(lambda x: payer_category(x))

In [39]:
df["code"] = df["code"].str.lstrip("'")

In [50]:
tin = file.split("_")[0]
tin = tin[:2] + "-" + tin[2:]

In [53]:
df["file_last_updated"] = updated_date
df["filename"] = file
df["url"] = url
df["hospital_ccn"] = ccn
df["hospital_tin"] = tin

In [54]:
df.to_csv('')

,procedure_desc,code,rev_code,rev_desc,hcpcs_cpt,code_type,billing_class,patient_class,billed_quantity,modifier,...,rate_method,rate_desc,payer_desc,rate,payer_category,file_last_updated,filename,url,hospital_ccn,hospital_tin
0,Liver Transplant And/Or Intestinal Transplant,001-1,NaN,NaN,NaN,apr-drg,facility,inpatient,NaN,NaN,...,apr-drg,NaN,payer_specific_negotiated_rate,"$35,821.79",rate,2023-03-19,860800150_mayo_head.csv,https://mcorgstatic.blob.core.windows.net/cms-...,030103,86-0800150
1,LIVER TRANSPLANT AND/OR INTESTINAL TRANSPLANT,001-1,NaN,NaN,NaN,apr-drg,facility,inpatient,NaN,NaN,...,apr-drg,NaN,payer_specific_negotiated_rate,"$55,579.42",rate,2023-03-19,860800150_mayo_head.csv,https://mcorgstatic.blob.core.windows.net/cms-...,030103,86-0800150
2,Liver Transplant And/Or Intestinal Transplant,001-1,NaN,NaN,NaN,apr-drg,facility,inpatient,NaN,NaN,...,apr-drg,NaN,standard_gross_charge,NaN,gross,2023-03-19,860800150_mayo_head.csv,https://mcorgstatic.blob.core.windows.net/cms-...,030103,86-0800150
3,LIVER TRANSPLANT AND/OR INTESTINAL TRANSPLANT,001-1,NaN,NaN,NaN,apr-drg,facility,inpatient,NaN,NaN,...,apr-drg,NaN,standard_gross_charge,NaN,gross,2023-03-19,860800150_mayo_head.csv,https://mcorgstatic.blob.core.windows.net/cms-...,030103,86-0800150
4,Liver Transplant And/Or Intestinal Transplant,001-1,NaN,NaN,NaN,apr-drg,facility,inpatient,NaN,NaN,...,apr-drg,NaN,discounted_cash_price,NaN,cash,2023-03-19,860800150_mayo_head.csv,https://mcorgstatic.blob.core.windows.net/cms-...,030103,86-0800150
5,LIVER TRANSPLANT AND/OR INTESTINAL TRANSPLANT,001-1,NaN,NaN,NaN,apr-drg,facility,inpatient,NaN,NaN,...,apr-drg,NaN,discounted_cash_price,NaN,cash,2023-03-19,860800150_mayo_head.csv,https://mcorgstatic.blob.core.windows.net/cms-...,030103,86-0800150
6,Liver Transplant And/Or Intestinal Transplant,001-1,NaN,NaN,NaN,apr-drg,facility,inpatient,NaN,NaN,...,apr-drg,NaN,minimum_negotiated,"$35,821.79",min,2023-03-19,860800150_mayo_head.csv,https://mcorgstatic.blob.core.windows.net/cms-...,030103,86-0800150
7,LIVER TRANSPLANT AND/OR INTESTINAL TRANSPLANT,001-1,NaN,NaN,NaN,apr-drg,facility,inpatient,NaN,NaN,...,apr-drg,NaN,minimum_negotiated,NaN,min,2023-03-19,860800150_mayo_head.csv,https://mcorgstatic.blob.core.windows.net/cms-...,030103,86-0800150
8,Liver Transplant And/Or Intestinal Transplant,001-1,NaN,NaN,NaN,apr-drg,facility,inpatient,NaN,NaN,...,apr-drg,NaN,maximum_negotiated,"$35,821.79",max,2023-03-19,860800150_mayo_head.csv,https://mcorgstatic.blob.core.windows.net/cms-...,030103,86-0800150
9,LIVER TRANSPLANT AND/OR INTESTINAL TRANSPLANT,001-1,NaN,NaN,NaN,apr-drg,facility,inpatient,NaN,NaN,...,apr-drg,NaN,maximum_negotiated,NaN,max,2023-03-19,860800150_mayo_head.csv,https://mcorgstatic.blob.core.windows.net/cms-...,030103,86-0800150


In [73]:
df.columns

Index(['procedure_desc', 'code', 'rev_code', 'rev_desc', 'hcpcs_cpt',
       'code_type', 'billing_class', 'patient_class', 'billed_quantity',
       'modifier', 'standard_gross_charge', 'discounted_cash_price',
       'payer_name', 'payer_specific_negotiated_rate', 'minimum_negotiated',
       'maximum_negotiated', 'rate_method', 'rate_desc'],
      dtype='object')